In [1]:
# Importing the required libraries
import os
import pandas as pd
from lxml import etree
from bs4 import BeautifulSoup
import random
import json
import time
import matplotlib as mpl
import matplotlib.pyplot as plt
from concurrent.futures import ThreadPoolExecutor
from concurrent.futures import ProcessPoolExecutor
# import multiprocessing as mp

# NLTK imports
import nltk

nltk.data.path.append('../nltk_data/')
import string
from nltk import collocations
from nltk.text import Text
from nltk.tokenize import WhitespaceTokenizer
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize

In [17]:
# load scraped data
df = pd.read_csv('../data/summary_20231127.csv')
df['date'] = pd.to_datetime(df['date'])
df = df[df['date'] > '2023-10-7']

In [18]:
import ast 
import re

rx = r"\.(?!(?<=\d\.)\d)(?=\S)"

# fix some of the full stop formatting in the original files
df['text'] = df['text'].apply(ast.literal_eval)
df['text'] = df['text'].apply(lambda x: re.sub(rx, ". ", ' '.join([s for s in x])))

In [19]:
def generate_article_files(row):
    text = ""
    
    _id = row['id']
    title = row['title']
    date = row['date'].date()
    content = row['text']
    
    # Assemble as part of large text block
    text += " F-N-S-T-A-R-T. "
    text += str(_id)
    text += " F-N-E-N-D. "
    text += " D-A-T-E-S-T-A-R-T. "
    text += date.strftime(format="%Y-%m-%d")
    text += " D-A-T-E-E-N-D. "
    text += " T-I-T-L-E-S-T-A-R-T. "
    text += title
    text += " T-I-T-L-E-E-N-D. "
    text += " T-E-X-T-S-T-A-R-T. "
    text += content
    text += " T-E-X-T-E-N-D. "
    
    # Save current text chunk
    article_filename = "articles/article_" + str(_id) + ".txt"
    results_filename = "results/article_"+str(_id)+".txt.json"
    
    with open(article_filename, 'w') as f:
        f.write(text)
        
    #create dataframe which points to expected files
    row['article_file'] = article_filename
    row['results_file'] = results_filename
    
    return row

# generate article file in the format required for stanford nlp
df = df.apply(generate_article_files,axis=1)

In [20]:
df.to_csv('./data/summary_20231127.csv', index=False)

In [31]:
# generate all-texts.txt which is going to be used for stanford nlp
import glob
all_articles = [os.path.abspath(f) for f in glob.glob("./articles/*.txt")]

with open("./all-files.txt", "w") as outfile:
    outfile.write("\n".join(all_articles))